In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 12.6MB/s 
     |████████████████████████████████| 2.9MB 56.5MB/s 
     |████████████████████████████████| 890kB 56.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7a3ae9710952d180ff85accab3166be102c05645dc071eed0d78a7b596171ec8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import os
os.chdir("/content/drive/My Drive/NLP_Fake_News")

In [ ]:
import torch
import pandas as pd
import numpy as np
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import torch.nn as nn
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
import random
import time
import os
import re

In [ ]:
!ls

answer.csv	       English_Train_2.gsheet  submit.csv
English_Test_2.gsheet  English_Train.gsheet    submit_test.csv
English_Test.gsheet    English_Val.gsheet      submit_val_to_check_bert.csv


In [ ]:
worksheet = gc.open('English_Train').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
train_df = pd.DataFrame.from_records(rows)

new_header = train_df.iloc[0] #grab the first row for the header
train_df = train_df[1:] #take the data less the header row
train_df.columns = new_header #set the header row as the df header

label_news = {'real': 0, 'fake': 1}
train_df['int_label'] = train_df['label'].map(label_news)
train_df = train_df.reset_index()

In [ ]:
train_df['tweet'][0]

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

In [ ]:
worksheet_val = gc.open('English_Val').sheet1

# get_all_values gives a list of rows.
rows_val = worksheet_val.get_all_values()
val_df = pd.DataFrame.from_records(rows_val)

val_new_header = val_df.iloc[0] #grab the first row for the header
val_df = val_df[1:] #take the data less the header row
val_df.columns = val_new_header #set the header row as the df header

label_news = {'real': 0, 'fake': 1}
val_df['int_label'] = val_df['label'].map(label_news)
val_df = val_df.reset_index()

In [ ]:
val_df['int_label'][0]

1

In [ ]:
worksheet_test = gc.open('English_Test_2').sheet1

# get_all_values gives a list of rows.
rows_test = worksheet_test.get_all_values()
test_df = pd.DataFrame.from_records(rows_test)

test_new_header = test_df.iloc[0] #grab the first row for the header
test_df = test_df[1:] #take the data less the header row
test_df.columns = test_new_header #set the header row as the df header
#label_news = {'real': 0, 'fake': 1}
test_df['int_label'] = list(range(1,len(test_df)+1))#test_df['label'].map(label_news)
test_df = test_df.reset_index()


In [ ]:
print(test_df)

0     index    id                                              tweet  int_label
0         1     1  Our daily update is published. States reported...          1
1         2     2             Alfalfa is the only cure for COVID-19.          2
2         3     3  President Trump Asked What He Would Do If He W...          3
3         4     4  States reported 630 deaths. We are still seein...          4
4         5     5  This is the sixth time a global health emergen...          5
...     ...   ...                                                ...        ...
2135   2136  2136  #CoronaVirusUpdates: State-wise details of Tot...       2136
2136   2137  2137  Tonight 12(midnight) onwards Disaster Manageme...       2137
2137   2138  2138  296 new cases of #COVID19Nigeria; Plateau-85 E...       2138
2138   2139  2139  RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...       2139
2139   2140  2140  More than half of pregnant women recently admi...       2140

[2140 rows x 4 columns]


In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [ ]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
class FakeNewsDataset(Dataset):
    """Fake News Dataset"""

    def __init__(self, df, tokenizer, MAX_LEN):
        """
        Args:
            csv_file (string): Path to the csv file with text and img name.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv_data = df
        self.tokenizer_bert = tokenizer
        self.MAX_LEN = MAX_LEN

    def __len__(self):
        return self.csv_data.shape[0]
    
    def pre_processing_BERT(self, sent):
        # Create empty lists to store outputs
        input_ids = []
        attention_mask = []
        
        encoded_sent = self.tokenizer_bert.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=self.MAX_LEN,                  # Max length to truncate/pad
            padding='max_length',         # Pad sentence to max length
            # return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      # Return attention mask
            truncation=True
            )
        
        input_ids = encoded_sent.get('input_ids')
        attention_mask = encoded_sent.get('attention_mask')
        
        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(attention_mask)
        
        return input_ids, attention_mask
     
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.csv_data['tweet'][idx]
        
        tensor_input_id, tensor_input_mask = self.pre_processing_BERT(text)

        label = self.csv_data['int_label'][idx]
        label = torch.tensor(label)
        sample = [tensor_input_id, tensor_input_mask, label]
        # print(idx)

        return sample

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Specify `MAX_LEN`
MAX_LEN = 64

# Run function `preprocessing_for_bert` on the dataset
transformed_dataset_train = FakeNewsDataset(train_df, tokenizer, MAX_LEN)

transformed_dataset_val = FakeNewsDataset(val_df, tokenizer, MAX_LEN)
transformed_dataset_test=FakeNewsDataset(test_df, tokenizer, MAX_LEN)

train_dataloader = DataLoader(transformed_dataset_train, batch_size=16,
                        shuffle=True, num_workers=0)

val_dataloader = DataLoader(transformed_dataset_val, batch_size=16,
                        shuffle=True, num_workers=0)
test_dataloader = DataLoader(transformed_dataset_test, batch_size=16,
                        shuffle=False, num_workers=0)


In [ ]:
for batch_idx, ans in enumerate(train_dataloader):
    print('Batch idx {}, dataset index {}'.format(
        batch_idx, ans[-1]))

Batch idx 0, dataset index tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0])
Batch idx 1, dataset index tensor([1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
Batch idx 2, dataset index tensor([1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0])
Batch idx 3, dataset index tensor([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0])
Batch idx 4, dataset index tensor([0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0])
Batch idx 5, dataset index tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1])
Batch idx 6, dataset index tensor([1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1])
Batch idx 7, dataset index tensor([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
Batch idx 8, dataset index tensor([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0])
Batch idx 9, dataset index tensor([1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0])
Batch idx 10, dataset index tensor([0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0])
Batch idx 11, dataset index tensor([0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 

In [ ]:
for x in train_dataloader:
    for t in x:
        print(t.shape)
        print(type(t))
    break

torch.Size([16, 64])
<class 'torch.Tensor'>
torch.Size([16, 64])
<class 'torch.Tensor'>
torch.Size([16])
<class 'torch.Tensor'>


In [ ]:
# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [ ]:
def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
# Specify loss function
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            # print(batch)
            # print([type(t) for t in batch])
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")

In [ ]:
def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
for batch_idx, ans in enumerate(train_dataloader):
    print('Batch idx {}, dataset index {}'.format(
        batch_idx, ans[-1]))

Batch idx 0, dataset index tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0])
Batch idx 1, dataset index tensor([0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1])
Batch idx 2, dataset index tensor([1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1])
Batch idx 3, dataset index tensor([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0])
Batch idx 4, dataset index tensor([1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1])
Batch idx 5, dataset index tensor([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0])
Batch idx 6, dataset index tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1])
Batch idx 7, dataset index tensor([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0])
Batch idx 8, dataset index tensor([1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1])
Batch idx 9, dataset index tensor([1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1])
Batch idx 10, dataset index tensor([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1])
Batch idx 11, dataset index tensor([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 

In [ ]:
## Training 
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=False)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.488730   |     -      |     -     |   3.09   
   1    |   40    |   0.269039   |     -      |     -     |   2.95   
   1    |   60    |   0.253583   |     -      |     -     |   2.94   
   1    |   80    |   0.250061   |     -      |     -     |   3.03   
   1    |   100   |   0.322083   |     -      |     -     |   2.93   
   1    |   120   |   0.212990   |     -      |     -     |   2.99   
   1    |   140   |   0.217332   |     -      |     -     |   2.97   
   1    |   160   |   0.198653   |     -      |     -     |   2.96   
   1    |   180   |   0.165036   |     -      |     -     |   2.97   
   1    |   200   |   0.242497   |     -      |     -     |   2.94   
   1    |   220   |   0.177028   |     -      |     -     |   2.91   
   1    |   240   |   0.150108   |     -      |     -     |   2.96   


In [ ]:
predictions=[]
val_accuracy=[]
for batch in test_dataloader:
  
  #print(batch)
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  print(batch)
  
  # Unpack the inputs from our dataloader
  b_ids , b_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs1 = bert_classifier(b_ids,b_mask)

  logits1 = outputs1
  logits2 = outputs1[0]

  
  
  # Store predictions and true labels
  preds=torch.argmax(logits1,dim=1).flatten()
  predictions.append(preds.cpu().numpy())

(tensor([[  101,  2256,  3679,  ...,     0,     0,     0],
        [  101, 22989, 10270,  ...,     0,     0,     0],
        [  101,  2343,  8398,  ...,     0,     0,     0],
        ...,
        [  101,  2019,  5746,  ...,  1997,  3572,   102],
        [  101,  2758,  1996,  ...,     0,     0,     0],
        [  101,  4268,  3362,  ...,     0,     0,     0]], device='cuda:0'), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       device='cuda:0'))


ValueError: ignored

In [ ]:
predictions[0].shape
finaloutput=np.concatenate(predictions,axis=0)

In [ ]:
df=pd.DataFrame(columns=['id','label'])
df['id']=list(range(1,len(finaloutput)+1))
df['label']=finaloutput.tolist()
df.to_csv("submit_test_2.csv",index=False)